# Importar Librerías

In [1]:
import datetime
import pickle
import polars as pl
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar Datos

In [2]:
cols_df1 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOUNITARIO',
    'CANTIDAD',
    'MONTORECLAMADO',
    'MONTOAJUSTADO',
    'MONTODIFERENCIAAFILIADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'TipoInstitucionPrestador'
    ]

cols_df2 = [
    'CLIENTEAFILIADOBENEFICIARIO',
    'AUTORIZACION',
    'FECHASERVICIO',
    'TIPOATENCION',
    'PRESTADOR',
    'TipoPrestador',
    'ESPECIALIDAD',
    'PRESTACION',
    'TIPOPRESTACION',
    'SERVICIOSALUD',
    'MONTOAJUSTADO',
    'MONTOCOPAGO',
    'AUTORIZACIONRELACIONADA',
    'MONTOASEGURADORA',
    'MONTOEXCEPCION',
    'MONTOCONCESION',
    'MONTOAUTORIZADO'
    ]


In [3]:
# Data Autorizaciones

df = pl.read_parquet(
    '../Datos/dataAutorizaciones20221208.parquet',
    columns = cols_df1
)

df = df.sample(500000)
df = df.to_pandas()

print(type(df))
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
(500000, 19)


In [4]:
# Data Distribuciones Autorizaciones

df2 = pl.read_parquet(
    '../Datos/dataDistribucionesAutorizaciones20221208.parquet',
    columns = cols_df2
)

df2 = df2.sample(500000)
df2 = df2.to_pandas()

print(type(df2))
print(df2.shape)


<class 'pandas.core.frame.DataFrame'>
(500000, 17)


In [5]:
# CUPS frecuentes en emergencias
cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx',engine='openpyxl'
)

# CUPS cantidades esperadas
valores_referencia_cups_df = pd.read_excel(
    '../Datos/CUPS frecuencias y otros (002).xlsx', 
    sheet_name=1,engine='openpyxl'
)

# Indicador de dias de habitación

# Read File
salas_df = pd.read_excel(
    '../Datos/salas.xlsx', engine='openpyxl'
)

salas_list = salas_df['DESCRIPCION'].tolist()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


# Unir Datasets

In [6]:
# Extraer autorizaciones en los dos datasets
autorizaciones_1 = set(df.AUTORIZACION.unique())
autoriaciones_2 = set(df2.AUTORIZACION.unique())
autorizaciones = autorizaciones_1.intersection(autoriaciones_2)

tmp_df_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO',
    'PRESTACION', 
    'PRESTADOR', 
    'FECHASERVICIO', 
    'MONTORECLAMADO'
]

tmp_df2_columns = [
    'AUTORIZACION', 
    'CLIENTEAFILIADOBENEFICIARIO', 
    'PRESTACION', 
    'PRESTADOR',
    'FECHASERVICIO', 
    'MONTOASEGURADORA', 
    'MONTOEXCEPCION', 
    'MONTOCONCESION', 
    'MONTOAUTORIZADO'
]

tmp_df = df[df.AUTORIZACION.isin(autorizaciones)][tmp_df_columns]
tmp_df2 = df2[df2.AUTORIZACION.isin(autorizaciones)][tmp_df2_columns]

# agrupar montos en el dataset de distribución
tmp_df2_new = tmp_df2.groupby(['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO', 'PRESTACION', 'PRESTADOR', 
                               'FECHASERVICIO'])[['MONTOASEGURADORA', 'MONTOEXCEPCION', 'MONTOCONCESION', 'MONTOAUTORIZADO']].sum().reset_index()

# Consolidar las dos bases de datos
consolidated_df = df.merge(
    tmp_df2_new,
    on=['AUTORIZACION', 'CLIENTEAFILIADOBENEFICIARIO','PRESTACION', 'PRESTADOR', 'FECHASERVICIO'],
)

consolidated_df.drop(columns=['MONTOASEGURADORA_x'], inplace=True)
consolidated_df.rename(
    columns={
        'MONTOASEGURADORA_y': 'MONTOASEGURADORA',
        'MONTODIFERENCIAAFILIADO': 'MONTOUSUARIO',
        'TipoPrestador': 'TIPOPRESTADOR',
    },
    inplace=True
)

In [7]:
del(df, df2, tmp_df, tmp_df2, tmp_df2_new)

In [8]:
consolidated_df.shape

(16381, 22)

# Preprocesamiento

In [21]:
df = consolidated_df.copy()

In [22]:
def clean_and_format_data(df):
    df['PRESTACION'] = df['PRESTACION'].str.strip() # Eliminar espacios al final y al comienzo de Prestacion
    df['FECHASERVICIO'] = pd.to_datetime(df['FECHASERVICIO']) # Dar formato a Fechas de Servicio
    df['FECHASERVICIO_MONTH'] = df['FECHASERVICIO'].dt.month_name() # Obtener mes de fecha de servicio
    df['FECHASERVICIO_MONTH_NUMBER'] = df['FECHASERVICIO'].dt.month
    return df.head()

In [23]:
clean_and_format_data(df)

,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,PRESTACION,TIPOPRESTACION,SERVICIOSALUD,...,MONTOUSUARIO,MONTOCOPAGO,AUTORIZACIONRELACIONADA,TipoInstitucionPrestador,MONTOASEGURADORA,MONTOEXCEPCION,MONTOCONCESION,MONTOAUTORIZADO,FECHASERVICIO_MONTH,FECHASERVICIO_MONTH_NUMBER
0,N-112696,48344159,2022-10-24,AMBULATORIA,LABORATORIO CLINICO AMADITA PITALUGA GONZALEZ,PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,"CREATININA EN SUERO, ORINA U OTROS",QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,24.0,0.0,NaN,LABORATORIO,136.0,0.0,0.0,136.0,October,10
1,N-570937,46810660,2022-05-13,AMBULATORIA,"FARMACIA YNA, C. POR A.",SUMNISTRADORES DE FARMACOS Y MEDICINAS,ATENCION FARMACEUTICA,Medicinas Ambulatorias,MEDICIMENTOS(TODAS SUS APLICACIONES),MEDICINAS AMBULATORIAS,...,264.0,0.0,NaN,FARMACIAS,396.0,0.0,0.0,396.0,May,5
2,N-2040831,46521223,2022-04-11,AMBULATORIA,"FARMACIA CAROL,SAN CRISTOBAL",SUMNISTRADORES DE FARMACOS Y MEDICINAS,ATENCION FARMACEUTICA,"DICLOFENACO POTASICO, TAB 50MG",MEDICAMENTOS AMBULATORIOS PDSS_3,MEDICINAS AMBULATORIAS,...,60.0,60.0,NaN,FARMACIAS,140.0,0.0,0.0,140.0,April,4
3,N-1645987,44933563,2021-10-22,AMBULATORIA,INSTITUTO CONTRA LA CEGUERA POR GLAUCOMA,"CLINICAS, HOSPITALES Y CENTROS MEDICOS",OFTALMOLOGIA,ESTUDIO DE CAMPO VISUAL CENTRAL Y PERIFERICO C...,EXAMENES DE OJOS GENERAL Y SUBJETIVO,ESTUDIOS ESPECIALIZADOS,...,300.0,400.0,NaN,CENTROS DIAGNÓSTICOS,1600.0,0.0,0.0,1600.0,October,10
4,N-1217522,45633693,2022-01-07,AMBULATORIA,MARIA AURORA SOSA SANCHEZ,MEDICOS Y/O ESPECIALISTAS,GINECOLOGIA Y OBSTETRICIA,CONSULTA DE PRIMERA VEZ POR MEDICINA ESPECIALI...,"\ENTREVISTA""",CONSULTAS MEDICAS CUALQUIER ESPECIALIDAD,...,100.0,100.0,NaN,MÉDICOS,500.0,0.0,0.0,500.0,January,1


In [26]:
def diferencia_montos(df):
    df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'] = df['MONTORECLAMADO'] - df['MONTOAUTORIZADO'] ### Calcular Diferencia de montos reclamado y autorizado
    df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO'] = df['MONTORECLAMADO'] - (df['MONTOAUTORIZADO'] + df['MONTOUSUARIO'])### Calcular Diferencia de montos reclamado y autorizado y usuario
    
    ### Calcular Valores Absolutos
    df['MONTOAJUSTADO_ABS'] = abs(df['MONTOAJUSTADO'])
    df['MONTOCONCESION_ABS'] = abs(df['MONTOCONCESION'])
    df['DIFERENCIA_ABSOLUTA_MONTO_RECLAMADO_AUTORIZADO'] = abs(df['DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO'])
    



diferencia_montos(df)  

In [27]:
df.head(3)

,CLIENTEAFILIADOBENEFICIARIO,AUTORIZACION,FECHASERVICIO,TIPOATENCION,PRESTADOR,TIPOPRESTADOR,ESPECIALIDAD,PRESTACION,TIPOPRESTACION,SERVICIOSALUD,...,MONTOEXCEPCION,MONTOCONCESION,MONTOAUTORIZADO,FECHASERVICIO_MONTH,FECHASERVICIO_MONTH_NUMBER,DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO,DIFERENCIA_MONTO_RECLAMADO_AUTORIZADO_USUARIO,MONTOAJUSTADO_ABS,MONTOCONCESION_ABS,DIFERENCIA_ABSOLUTA_MONTO_RECLAMADO_AUTORIZADO
0,N-112696,48344159,2022-10-24,AMBULATORIA,LABORATORIO CLINICO AMADITA PITALUGA GONZALEZ,PRESTADORES DE SERVICIOS DE APOYO DIAGNOSTICOS,LABORATORIO CLINICO DE HEMATOLOGIA Y BAN,"CREATININA EN SUERO, ORINA U OTROS",QUIMICA SANGUINEA Y DE OTROS FLUIDOS CORPORALES,LABORATORIOS,...,0.0,0.0,136.0,October,10,24.0,0.0,0.0,0.0,24.0
1,N-570937,46810660,2022-05-13,AMBULATORIA,"FARMACIA YNA, C. POR A.",SUMNISTRADORES DE FARMACOS Y MEDICINAS,ATENCION FARMACEUTICA,Medicinas Ambulatorias,MEDICIMENTOS(TODAS SUS APLICACIONES),MEDICINAS AMBULATORIAS,...,0.0,0.0,396.0,May,5,264.0,0.0,0.0,0.0,264.0
2,N-2040831,46521223,2022-04-11,AMBULATORIA,"FARMACIA CAROL,SAN CRISTOBAL",SUMNISTRADORES DE FARMACOS Y MEDICINAS,ATENCION FARMACEUTICA,"DICLOFENACO POTASICO, TAB 50MG",MEDICAMENTOS AMBULATORIOS PDSS_3,MEDICINAS AMBULATORIAS,...,0.0,0.0,140.0,April,4,60.0,0.0,0.0,0.0,60.0


In [30]:
def calcular_fechas_1(df, salas_list):
    
    # Calcular Dias de habitación
    dias_habitacion_mask = df['PRESTACION'].isin(salas_list)
    df.loc[dias_habitacion_mask, 'DIAS_HABITACION'] = df['CANTIDAD']
    
    # Calcular la fecha de fin de servicio
    def calculate_end_date(row):
        if np.isnan(row['DIAS_HABITACION']):
            return np.nan
        else:
            start_date = row['FECHASERVICIO']
            dias_habitacion = row['DIAS_HABITACION']
            end_date = start_date + datetime.timedelta(days=dias_habitacion)
            return end_date
        
    df['FECHASERVICIO_FIN'] = df.apply(calculate_end_date, axis=1)
    

calcular_fechas_1(df, salas_list)

In [31]:
print(df.shape)
print(consolidated_df.shape)

(16381, 31)
(16381, 22)


In [32]:
# Alertas para Reglas de Negocio, relacionada a montos

especialidades_claves = [
    'CENTROS MEDICOS',
    'ATENCION FARMACEUTICA', 
    'CTROS. DIAGNOSTICOS',
    'MEDICINA FISICA Y REHABILITACION', 
    'LABORATORIO CLINICO DE HEMATOLOGIA Y BAN',
]

def tipo_especialidad(df, especialidades_claves):
    df['TIPO_ESPECIALIDAD'] = df['ESPECIALIDAD']
    df.loc[~df['TIPO_ESPECIALIDAD'].isin(especialidades_claves), 'TIPO_ESPECIALIDAD'] = 'OTROS'
    

tipo_especialidad(df, especialidades_claves)

In [33]:
print(df.shape)
print(consolidated_df.shape)

(16381, 32)
(16381, 22)
